In [45]:
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import torch

In [2]:
# class MyClass(Dataset):
#     def __init__(self):
#         self.df = pd.read_csv('./data/movies.csv')
#     def __getitem__(self,index):
#         return self.df.iloc[index]
#     def __len__(self):
#         return len(self.df)
# my_dataset = MyClass()
# print(my_dataset[1])

In [3]:
# dataset = DataLoader(dataset=my_dataset, batch_size=206, shuffle=True, num_workers=4)
# len(dataset)

In [4]:
from torchvision.datasets import MNIST

In [5]:
mist = MNIST('./data',train=True, download=False, transform=None)

In [6]:
print(mist)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train


In [15]:
mist[1][0]

In [25]:
from torchvision.transforms import Compose,Normalize,ToTensor
from torch.utils.data import DataLoader
transform_fn = Compose([
    ToTensor(),
    Normalize(mean=(0.1307,),std=(0.3081,))
])
dataset = MNIST('./data',train=True, download=False, transform=transform_fn)
# data_loader = DataLoader(dataset,batch_size=12,shuffle=True)
# list(data_loader)

In [26]:
import torch.nn as nn
import torch.nn.functional as F
class MinistMoudle(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1*28*28, 28)
        self.fc2 = nn.Linear(28, 10)
    def forward(self,input):
        x = input.view([-1,1*28*28])
        x = self.fc1(x)
        x = F.relu(x)
        out = self.fc2(x)
        return F.log_softmax(out)

In [62]:
from torch.optim import Adam
model = MinistMoudle()
optimizer = Adam(model.parameters(),lr=0.001)
def train(epoch):
    data_loader = DataLoader(dataset,batch_size=2,shuffle=True)
    for index,(input,target) in enumerate(data_loader):
        output = model(input)
        loss = F.nll_loss(output,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if index%100==0:
            print(loss)
            # 模型保存
            torch.save(model.state_dict(),"./modle")
            torch.save(optimizer.state_dict(),"./optimizer")

In [76]:
import numpy as np
def test():
    dataset = MNIST('./data',train=True, download=False, transform=transform_fn)
    data_loader = DataLoader(dataset,batch_size=12,shuffle=True)
    loss_list = []
    acc_list = []
    for index,(input,target) in enumerate(data_loader):
        with torch.no_grad():
            output = model(input)
            loss = F.nll_loss(output,target)
            loss_list.append(loss)
            pred = output.max(dim=-1)[-1] # 回去最大值位置
            cur_acc = pred.eq(target).float().mean()
            acc_list.append(cur_acc)
    print(np.mean(loss_list),np.mean(acc_list))

In [77]:
if __name__=='__main__':
    test()

/var/folders/j9/x9wd0pm15rgd97b0jsp8sk100000gn/T/ipykernel_7400/3120936917.py:13: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(out)


0.19650857 0.94
